In [1]:
import tensorflow as tf
import keras
import numpy as np
%matplotlib inline
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Input, Flatten, Dropout, AveragePooling2D
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
from glob import glob
import cv2

In [3]:
train = np.array(glob("training/1/*"))

In [4]:
X_train = []
y_train = []
for img_path in train:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(50,10))
    X_train.append(img)
    y_train.append(0)

In [5]:
train = np.array(glob("training/2/*"))
for img_path in train:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(50,10))
    X_train.append(img)
    y_train.append(1)

In [6]:
train = np.array(glob("training/3/*"))
for img_path in train:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(50,10))
    X_train.append(img)
    y_train.append(2)

In [7]:
train = np.array(glob("training/4/*"))
for img_path in train:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(50,10))
    X_train.append(img)
    y_train.append(3)

In [8]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)

y_train = keras.utils.to_categorical(y_train, 4)

In [9]:
print(X_train.shape)
print(y_train.shape)

(3715, 10, 50, 1)
(3715, 4)


In [10]:
def shuffle(x,y):
    indices = np.arange(x.shape[0])
    rand_indices = np.random.shuffle(indices)
    x[indices] = x[rand_indices]
    y[indices] = y[rand_indices]
    return x,y

In [11]:
X_train, y_train = shuffle(X_train, y_train)
X_test = X_train[3000:]
y_test = y_train[3000:]
X_train = X_train[:3000]
y_train = y_train[:3000]

In [12]:
def FaceModel(input_shape = (10, 50, 1)):
    X_input = Input(input_shape)
    X = Conv2D(filters = 10, kernel_size = (5, 5), activation='relu', padding='same')(X_input)
    X = AveragePooling2D(pool_size=(2, 2))(X)
    
    X = Conv2D(filters = 10, kernel_size = (5, 5), activation='relu', padding='same')(X)
    X = MaxPooling2D(pool_size= (2, 2))(X)
    
    X = Conv2D(filters = 10, kernel_size = (3, 3), activation='relu', padding='same')(X)
    X = MaxPooling2D(pool_size= (2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(256, activation = 'relu')(X)
    X = Dropout(0.5)(X)
    
    X = Dense(128, activation = 'relu')(X)
    X = Dropout(0.5)(X)
    
    X = Dense(4, activation = 'softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='FaceModel')

    return model

In [13]:
model = FaceModel()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 50, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 50, 10)        260       
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 25, 10)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 25, 10)         2510      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 12, 10)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 12, 10)         910       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 6, 10)          0         
__________

In [14]:
model.compile(optimizer= 'adam' , loss = 'categorical_crossentropy',
              metrics= ['accuracy'])
#test_generator = test_datagen.flow(X_test, y_test)

In [15]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:

train_generator = train_datagen.flow(X_train, y_train, batch_size=64)
test_generator = test_datagen.flow(X_test, y_test, batch_size=64)


In [17]:
model.fit_generator(train_generator,
                            steps_per_epoch=X_train.shape[0]//64,
                            epochs=20,validation_data=test_generator,
                            validation_steps=20)

Epoch 1/20
46/46 [==============================] - 7s 146ms/step - loss: 1.1400 - acc: 0.5481 - val_loss: 1.0018 - val_acc: 0.5844
Epoch 2/20
46/46 [==============================] - 1s 24ms/step - loss: 0.9364 - acc: 0.6386 - val_loss: 0.6903 - val_acc: 0.7637
Epoch 3/20
46/46 [==============================] - 1s 26ms/step - loss: 0.6673 - acc: 0.7600 - val_loss: 0.4943 - val_acc: 0.8060
Epoch 4/20
46/46 [==============================] - 1s 26ms/step - loss: 0.5015 - acc: 0.8311 - val_loss: 0.3660 - val_acc: 0.8883
Epoch 5/20
46/46 [==============================] - 1s 26ms/step - loss: 0.3841 - acc: 0.8764 - val_loss: 0.2704 - val_acc: 0.9161
Epoch 6/20
46/46 [==============================] - 1s 27ms/step - loss: 0.3237 - acc: 0.8950 - val_loss: 0.2032 - val_acc: 0.9324
Epoch 7/20
46/46 [==============================] - 1s 27ms/step - loss: 0.2576 - acc: 0.9188 - val_loss: 0.1780 - val_acc: 0.9438
Epoch 8/20
46/46 [==============================] - 1s 27ms/step - loss: 0.2334 - 

In [18]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt

In [19]:
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [ ]:
cap=cv2.VideoCapture(0)
flag=0
co = 836
while True:
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        mouth = shape[lStart:lEnd]
        x_min = min(mouth[:,1])
        x_max = max(mouth[:,1])
        y_min = min(mouth[:,0])
        y_max = max(mouth[:,0])
        inp_img = frame[x_min:x_max, y_min:y_max,:]
        inp_img = cv2.cvtColor(inp_img, cv2.COLOR_BGR2GRAY)
        inp_img = cv2.resize(inp_img,(50,10))
        inp_img = inp_img.reshape(1,inp_img.shape[0],inp_img.shape[1],1)
        print(np.argmax(model.predict(inp_img)))
        
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
0
1
1
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
3
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
2
0
0
2
2
2
0
1
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
0
0
1
0
0
0
0
0
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
